<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/8LLM_v2_module_5_large_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers openai unstructured
!pip install plotly
!pip install -U langchain-openai
!pip install langchain_community
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
!pip install tiktoken
!pip install matplotlib
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb gradio faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
!pip install linkify-it-py

In [ ]:
import numpy as np
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
import torch
import sys
import csv

csv.field_size_limit(sys.maxsize)

# torch.has_mps

131072

In [ ]:
# os.environ["OPENAI_API_KEY"] = getpass("")
#os.environ["OPENAI_API_KEY"] = “API KEY”

In [ ]:

if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

AssertionError: This doesn't look like a valid OpenAI API key

#Chunking and RAG

In [ ]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
# from langchain.document_loaders import TextLoader
# from langchain.chat_models import ChatOpenAI
# from langchain import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain
# from langchain.llms import OpenAI
# from langchain.vectorstores import FAISS

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
MODEL_NAME = "text-embedding-3-small"
# MODEL_NAME = "gpt-4"

In [ ]:
# from langchain.document_loaders import DirectoryLoader

# def find_md_files(directory):
#     "Find all markdown files in a directory and return a LangChain Document"
#     dl = DirectoryLoader(directory, "**/*.txt")
#     return dl.load()

# documents = find_md_files('data')
# len(documents)

In [ ]:
# documents

#upload files

In [ ]:
import pandas as pd

In [ ]:
# from langchain.document_loaders.csv_loader import CSVLoader
# # from langchain.docstore.document import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import DirectoryLoader

In [ ]:
url='https://drive.google.com/file/d/1gl7WAkJr6Nyke7YckzXxdL-iM4UjhLGX/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [ ]:
df.head()

,_id,title,author,description,publish_date,full_text,url,categories
0,63927168098523fa9d5ef047,Are The Democrats Screwed In The Senate After ...,Nate Silver,\nAre The Democrats Screwed In The Senate Afte...,1.670516e+12,\n\n\n\n2028 Election\nAre The Democrats Screw...,https://fivethirtyeight.com/features/democrats...,['news']
1,652b21ec754471a9795c291a,Beyond the Narrative,Sam Freedman,NaN,1.697272e+12,We are told before every party conference that...,https://samf.substack.com/p/beyond-the-narrative,['news']
2,6529cfaff59ed61396db838c,Ballot Measures: A Preview,Walter Olson,NaN,1.697227e+12,Walter Olson Voters will go to the polls soon...,https://www.cato.org/blog/ballot-measures-preview,['news']
3,6529d030f59ed61396db8398,Trump’s Only Real Worldview Is Pettiness,David A. Graham,NaN,1.697214e+12,Let no one say that Donald Trump has lost his ...,"tag:theatlantic.com,2023:50-675637",['news']
4,6529d072f59ed61396db839e,The COVID Bailout of State and Local Governmen...,Eric Boehm,NaN,1.697212e+12,Two years after Congress authorized a h...,https://reason.com/2023/10/13/the-covid-bailou...,['news']


In [ ]:
df.shape

(3327, 8)

In [ ]:
!mkdir data

In [ ]:
df[:100].to_csv('data/df_embed.csv',index=False)

1. Step 1: Read the data

In [ ]:
df.full_text	[1]

'We are told before every party conference that the leader will be making perhaps their “most important speech yet”. In reality, unless they are going into the conference with the word “beleaguered” regularly appearing in front of their name, they’ll almost certainly be fine. For struggling leaders a bad speech might sink them, but if they are 20 points ahead in the polls, and coming off a huge by-election win, then they are unlikely to be derailed.It’s even less likely to be a problem if your opponents are noisily falling apart. I wasn’t at the Tory conference but all of the several dozen journalists, politicians and business people I spoke to who had, were unanimous that it was even more shambolic than it appeared on TV. Even hacks from right-wing papers were shaking their heads at the sheer incoherence of it all. It’s hard to be a client journalist when your political masters can’t figure out what they want you to say.You can see how the Tories got into such a mess. There is no reco

In [ ]:
df.head()

,_id,title,author,description,publish_date,full_text,url,categories
0,63927168098523fa9d5ef047,Are The Democrats Screwed In The Senate After ...,Nate Silver,\nAre The Democrats Screwed In The Senate Afte...,1.670516e+12,\n\n\n\n2028 Election\nAre The Democrats Screw...,https://fivethirtyeight.com/features/democrats...,['news']
1,652b21ec754471a9795c291a,Beyond the Narrative,Sam Freedman,NaN,1.697272e+12,We are told before every party conference that...,https://samf.substack.com/p/beyond-the-narrative,['news']
2,6529cfaff59ed61396db838c,Ballot Measures: A Preview,Walter Olson,NaN,1.697227e+12,Walter Olson Voters will go to the polls soon...,https://www.cato.org/blog/ballot-measures-preview,['news']
3,6529d030f59ed61396db8398,Trump’s Only Real Worldview Is Pettiness,David A. Graham,NaN,1.697214e+12,Let no one say that Donald Trump has lost his ...,"tag:theatlantic.com,2023:50-675637",['news']
4,6529d072f59ed61396db839e,The COVID Bailout of State and Local Governmen...,Eric Boehm,NaN,1.697212e+12,Two years after Congress authorized a h...,https://reason.com/2023/10/13/the-covid-bailou...,['news']


In [ ]:
path= 'data/df_embed.csv'
loader = CSVLoader(file_path=path,source_column="title")

data = loader.load()

In [ ]:
print(data[0])

page_content='_id: 63927168098523fa9d5ef047
title: Are The Democrats Screwed In The Senate After 2024?
author: Nate Silver
description: Are The Democrats Screwed In The Senate After 2024?
No, but the party faces an uphill battlee


I’m not that impressed by long-term projections of Democratic doom in the Senate. Mostly that’s because I’m not that impressed by long-term political projections in general. Political coalitions change; not many people would have had it on their bingo card that Democrats would obtain a Senate majority in 2020 by winning two runoffs in Georgia, and then would actually expand on that majority in 2022 following another runoff in Georgia despite their party controlling the White House.
With that said, while I don’t think we can say very much about what politics will look like 10 or 20 years from now, a medium-term look-ahead is possible. Do Democrats have a reasonable hope of regaining a trifecta — control of the House, Senate and presidency — at any point in th

In [ ]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} words in your first document')

You have 100 document(s) in your data
There are 23887 words in your first document


2. Breakdown data - Chunking Process

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)

In [ ]:
len(texts)

1192

In [ ]:
print (f'You now have {len(texts)} document(s) in your data')
print (f'There are {len(texts[1].page_content)} characters in your document')

You now have 1192 document(s) in your data
There are 951 characters in your document


In [ ]:
texts[0]

Document(metadata={'source': 'Are The Democrats Screwed In The Senate After 2024?', 'row': 0}, page_content='_id: 63927168098523fa9d5ef047\ntitle: Are The Democrats Screwed In The Senate After 2024?\nauthor: Nate Silver\ndescription: Are The Democrats Screwed In The Senate After 2024?\nNo, but the party faces an uphill battlee')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: API KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
db.save_local("faiss_index")


#The new place to begin

In [ ]:
db_loaded = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [ ]:
query = """Why is it so bad for Democrats in Senate in 2024"""

In [ ]:
def get_response_from_query(db, query, k=10):
    """

    """

    docs = db.similarity_search(query, k=k) # extremely important

    docs_page_content = " ".join([d.page_content for d in docs])

    # llm = BardLLM()
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0)

    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        You are a bot that is open to discussions about different cultural, philosophical and political exchanges. I will use do different analysis to the articles provided to me. Stay truthful and if you weren't provided any resources give your oppinion only.
        Answer the following question: {question}
        By searching the following articles: {docs}

        Only use the factual information from the documents. Make sure to mention key phrases from the articles.

        If you feel like you don't have enough information to answer the question, say "I don't know".

        """,
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    # chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, prompt=prompt,
    #                                                     chain_type="stuff", retriever=db.as_retriever(), return_source_documents=True)

    response = chain.run(question=query, docs=docs_page_content,return_source_documents=True)
    r_text = str(response)

    ##evaluation part

    prompt_eval = PromptTemplate(
        input_variables=["answer", "docs"],
        template="""
        You job is to evaluate if the response to a given context is faithful.

        for the following: {answer}
        By searching the following article: {docs}

       Give a reason why they are similar or not, start with a Yes or a No.

        """,
    )

    chain_part_2 = LLMChain(llm=llm, prompt=prompt_eval)

    evals = chain_part_2.run(answer=r_text, docs=docs_page_content)


    return response,docs,evals

In [ ]:
answer,sources,evals=get_response_from_query(db_loaded,query,15)

In [ ]:
sources

NameError: name 'sources' is not defined

In [ ]:
from pprint import pprint

# Print question, answer, and evaluations
print("\n\n> Question:")
pprint(query)
print("\n> Answer:")
pprint(answer)
print("\n> Eval:")
pprint(evals)

# Print the relevant sources used for the answer
print("----------------------------------SOURCE DOCUMENTS---------------------------")
for document in sources:
    print("\n> " + document.metadata["source"])
    pprint(document.page_content[:1000])
print("----------------------------------SOURCE DOCUMENTS---------------------------")




> Question:
'Why is it so bad for Democrats in Senate in 2024'

> Answer:
('Based on the articles provided, it is suggested that the 2024 Senate map is '
 'unfavorable for Democrats, making it difficult for them to maintain control '
 'of the Senate. Losing key states such as West Virginia, Montana, and Ohio '
 'could result in Democrats having only 48 Senate seats. The articles also '
 'mention that Democrats have a narrow path to Senate control after 2024 and '
 'that it might require the GOP to nominate bad candidates and some luck for '
 'Democrats to regain control. The articles also discuss the potential '
 'retirement of liberal justices and the importance of Democrats limiting the '
 'damage in 2024 to have a chance of regaining the Senate in 2026 or 2028. '
 'Overall, the articles indicate that the 2024 Senate map poses challenges for '
 'Democrats in maintaining and regaining control of the Senate.')

> Eval:
('Yes, the response is faithful to the given context. The respons

In [ ]:
import gradio as gr

In [ ]:
def get_response_from_query(db, query, k=3):


    docs = db.similarity_search(query, k=k)

    docs_page_content = " ".join([d.page_content for d in docs])

    # llm = BardLLM()
    llm = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        A bot that is open to discussions about different cultural, philosophical and political exchanges. I will use do different analysis to the articles provided to me. Stay truthful and if you weren't provided any resources give your oppinion only.
        Answer the following question: {question}
        By searching the following articles: {docs}

        Only use the factual information from the documents. Make sure to mention key phrases from the articles.

        If you feel like you don't have enough information to answer the question, say "I don't know".

        """,
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    # chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, prompt=prompt,
    #                                                     chain_type="stuff", retriever=db.as_retriever(), return_source_documents=True)

    response = chain.run(question=query, docs=docs_page_content,return_source_documents=True)
    r_text = str(response)

    ##evaluation part

    prompt_eval = PromptTemplate(
        input_variables=["answer", "docs"],
        template="""
       You job is to evaluate if the response to a given context is faithful.

        for the following: {answer}
        By searching the following article: {docs}

       Give a reason why they are similar or not, start with a Yes or a No.
        """,
    )

    chain_part_2 = LLMChain(llm=llm, prompt=prompt_eval)


    evals = chain_part_2.run(answer=r_text, docs=docs_page_content)

    return response,docs,evals

In [ ]:
def greet(query):

    answer,sources,evals = get_response_from_query(db,query,2)
    return answer,sources,evals
examples = [
    ["How to be happy"],
    ["Climate Change Challenges in Europe"],
    ["Philosophy in the world of Minimalism"],
    ["Hate Speech  vs Freedom of Speech"],
    ["Articles by Noam Chomsky on US Politics"],
    ["The importance of values and reflection"]
    ]
demo = gr.Interface(fn=greet, title="cicero-semantic-search", inputs="text",
                    outputs=[gr.components.Textbox(lines=3, label="Response"),
                             gr.components.Textbox(lines=3, label="Source"),
                             gr.components.Textbox(lines=3, label="Evaluation")],
                   examples=examples)

demo.launch(share=True)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://98c6e16a1d66c362d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from gradio_client import Client

client = Client("https://91a238021b8734d96f.gradio.live/")
result = client.predict(
		"Can you tell me about the concerns of Democrats for 2024 Senate",	# str  in 'query' Textbox component
		api_name="/predict"
)
print(result)

Loaded as API: https://91a238021b8734d96f.gradio.live/ ✔
("Based on the provided articles, the concerns of Democrats for the 2024 Senate seem to revolve around the challenging electoral map that they will face. The articles mention that even with an additional senator going into 2023, the 2024 map is still unfavorable for Democrats, making it difficult for them to keep the Senate for years to come. The party's prospects may rely on limiting the damage in 2024 to have a chance of regaining the Senate in 2026 or 2028. However, a bad outcome in 2024 could make it very difficult for Democrats to regain the Senate before 2030 or 2032.\n\nIn summary, the concerns of Democrats for the 2024 Senate primarily stem from the unfavorable electoral map and the potential long-term implications if they are unable to limit the damage in that election.", "[Document(page_content='Even with an additional senator going into 2023, the 2024 map is still so bad for Democrats that keeping the Senate for years 